In [2]:
"""Imports"""

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys

In [12]:
"""Read CSV data into Dataframe"""
rawData = pd.read_csv('./data/EV_Population_WA_Data.csv')

# Drop irrelevant columns
data = pd.DataFrame([rawData['Make'], rawData['Model'], rawData['Model Year'], rawData['Electric Range'], rawData['City'], rawData['County']]).transpose()
print(data.head())
# cleanData = dataCleaner(data)
# [data['Electric Range'].gt(0)] #TODO: build function to actually clean the data

    Make    Model Model Year Electric Range          City     County
0  TESLA  MODEL 3       2018            215       Olympia   Thurston
1   AUDI   E-TRON       2021            222  Port Orchard     Kitsap
2  TESLA  MODEL 3       2018            215        Yakima     Yakima
3  TESLA  MODEL 3       2022              0        Yakima     Yakima
4  TESLA  MODEL Y       2023              0       Bothell  Snohomish


In [80]:
"""Dat Cleaning Function"""

knownRanges = data[data['Electric Range'].gt(0)].groupby(['Make', 'Model'])
unknownRanges = data[data['Electric Range'].lt(1)]

print('Starting Unknown Ranges', len(unknownRanges))

unrecoredRanges = []
def dataCleaner(data: pd.DataFrame):

    def getRange(record):
        if record['Electric Range'] == 0:
            recordedRange = 0 #TODO: this value may be better served as something else. 
            try:
                # print(knownRanges.get_group((record['Make'], record['Model']))['Electric Range'].mean)
                recordedRange = knownRanges.get_group((record['Make'], record['Model']))['Electric Range'].mean()
            except KeyError as e:
                # print('Encounted make x model combination with no recorded range available:', record['Make'], record['Model'])
                unrecoredRanges.append((record['Make'], record['Model']))
                # print('KEY ERROR!!!', str(e))
            
            record['Electric Range'] = recordedRange


    data.apply(getRange, axis=1)
    
    # unknownRanges.assign(Electric_Range = lambda record: knownRanges.get_group((record['Make'], record['Model']))['Electric Range'].mean())

dataCleaner(data)
print('Ending Unknown Ranges', len(data[data['Electric Range'].lt(1)]))

print(data[data['Electric Range'].lt(1)])

Starting Unknown Ranges 50093
Ending Unknown Ranges 38272


In [85]:
with open("output.txt", "a") as f:
    f.write(data[data['Electric Range'].lt(1)].groupby(['Make', 'Model']).first().to_csv())
    f.close()


In [4]:
"""Which car manufacturers are the most commonly used for EVs in Washington?"""

groupedByMake = cleanData.groupby('Make')

print(groupedByMake.size().sort_values(ascending=False)[:10])





Make
TESLA        25583
NISSAN       10677
CHEVROLET     9707
TOYOTA        6937
BMW           5811
JEEP          5288
KIA           4139
FORD          3985
CHRYSLER      3620
VOLVO         3182
dtype: int64


In [5]:
"""What are the highest and lowest electric ranges in this dataset, and which car
makers and models do they correspond to?"""

# Clean range data:
# makeModelRangeFix = pd.DataFrame([data['Make'], data['Model'], data['Model Year'], data['Electric Range']]).transpose()


# TODO: Update the below statement to exclude model duplicates (take mean of listed ranges)
knownRanges = cleanData[cleanData['Electric Range'].gt(0)].drop_duplicates()
ranges = knownRanges.groupby(['Make', 'Model'])

"""Using the .mean() function on the groupby results allows us to normalize for variances in driving styles and self reporting"""
meanMax = ranges['Electric Range'].mean().max()
meanMin = ranges['Electric Range'].mean().min()
# print(ranges['Electric Range'].mean())
print(ranges.get_group(('AUDI', 'A3'))['Electric Range'].mean())
# print('Maximum recorded range:', meanMax)
# print('Minimum recorded range:', meanMin)

# """Note: There are N make model pairings that we do not have range values for, this info will need to be supplied from supplementary sources. but for now will be ignored when claciulating"""
# # rangeToFix = makeModelRangeFix[makeModelRangeFix['Electric Range'].lt(1)].drop_duplicates()
# # print(len(rangeToFix))

# maxRange = ranges['Electric Range'].max().max()
# minRange = ranges['Electric Range'].min().min()

# print('Maximum recorded range:', maxRange)
# print('Minimum recorded range:', minRange)


16.0


In [307]:
"""Is the maximum electric range value unique? If not, which cars share this range?"""


maxRange = ranges['Electric Range'].max().sort_values(ascending=False)[:10]
print(maxRange)

"""
Multiple cars have the same recoded max range, however they are all the same make and model.
"""



Make       Model   
TESLA      MODEL S     337
           MODEL 3     322
           MODEL X     293
           MODEL Y     291
CHEVROLET  BOLT EV     259
HYUNDAI    KONA        258
TESLA      ROADSTER    245
KIA        NIRO        239
JAGUAR     I-PACE      234
POLESTAR   PS2         233
Name: Electric Range, dtype: object


'\nMultiple cars have the same recoded max range, however they are all the same make and model.\n'

In [309]:
"""Is the minimum electric range value unique? If not, which cars share this range?"""
minModels = ranges['Electric Range'].min().sort_values(ascending=True)[:10]



print('Min Range: \n', minModels)



Min Range: 
 Make           Model        
TOYOTA         PRIUS PLUG-IN     6
MERCEDES-BENZ  GLE-CLASS         8
               C-CLASS           8
               GLC-CLASS         9
PORSCHE        CAYENNE          12
MINI           COUNTRYMAN       12
MERCEDES-BENZ  S-CLASS          12
PORSCHE        918              12
HONDA          ACCORD           13
VOLVO          XC90             13
Name: Electric Range, dtype: object


In [304]:
"""How does the electric range vary between car makers?"""

Makes = knownRanges['Make'].unique()

Ranges = []

for make in Makes:
    range = knownRanges['Electric Range'].where(knownRanges['Make'] == make).mean()
    Ranges.append(range)

makeRange = pd.DataFrame(zip(Makes, Ranges), columns=['Make', 'Mean Range']).sort_values(by='Mean Range', ascending=False)
print(makeRange)


                    Make  Mean Range
0                  TESLA  247.879148
15                JAGUAR  234.000000
28              POLESTAR  233.000000
5              CHEVROLET  117.867337
6                 NISSAN  109.766475
12            VOLKSWAGEN  104.140000
33                 TH!NK  100.000000
34  WHEEGO ELECTRIC CARS  100.000000
3                   FIAT   85.438871
11               HYUNDAI   81.401667
1                   AUDI   75.730412
7                    KIA   72.737941
8                  SMART   62.459119
26               PORSCHE   61.474026
31        AZURE DYNAMICS   56.000000
2                    BMW   46.097384
16                 HONDA   45.703180
25                  MINI   41.335294
24                 LEXUS   36.839196
22              CADILLAC   36.420290
27         MERCEDES-BENZ   34.750000
32                FISKER   33.000000
21            ALFA ROMEO   33.000000
17              CHRYSLER   32.257931
14            MITSUBISHI   32.007792
23                 DODGE   32.000000
1

In [147]:
"""How does the electric range vary between maker models?"""

Makes = knownRanges['Make'].unique()

makeModelRange = {}

for make in Makes:
    makeModel = knownRanges[knownRanges['Make'] == make].sort_values(by='Electric Range', ascending=False)
    print(make, 'model with max range:\n', makeModel[:1])
    print(make, 'model with min range:\n', makeModel[-1:])
    print('\n')
    # A box plot to show range and identify outliers would be useful.
    

# makeRange = pd.DataFrame(zip(Makes, Ranges), columns=['Make', 'Mean Range']).sort_values(by='Mean Range', ascending=False)
# print(makeRange)


TESLA model with max range:
        Make    Model Model Year Electric Range
1517  TESLA  MODEL S       2020            337
TESLA model with min range:
       Make    Model Model Year Electric Range
627  TESLA  MODEL X       2016            200


AUDI model with max range:
    Make   Model Model Year Electric Range
1  AUDI  E-TRON       2021            222
AUDI model with min range:
      Make Model Model Year Electric Range
476  AUDI    A3       2017             16


BMW model with max range:
       Make Model Model Year Electric Range
42375  BMW    I3       2021            153
BMW model with min range:
      Make Model Model Year Electric Range
1226  BMW    X5       2018             13


FIAT model with max range:
     Make Model Model Year Electric Range
47  FIAT   500       2014             87
FIAT model with min range:
        Make Model Model Year Electric Range
16500  FIAT   500       2019             84


FORD model with max range:
       Make  Model Model Year Electric Range
11

In [148]:
"""Which are the top 5 cities adopting EVs?"""

# Note that this statistic is better served by calculating EV per capita, however the data set does not include population data for the given cities.
print(data['City'].value_counts()[:5])


City
Seattle      32602
Bellevue      9960
Redmond       7165
Vancouver     7081
Bothell       6602
Name: count, dtype: int64


In [161]:
"""How does the EV adoption rate vary among car makers over the years?"""
# Models per year by Make
byYear = pd.DataFrame([data['Make'], data['Model'], data['Model Year']]).transpose().drop_duplicates().sort_values(by=['Make', 'Model', 'Model Year'], ascending=[True, True, False])


"""we can express maker adoption rate by examining how many EV models a given manufacture produced over the years"""
for make in Makes:
    makeYear = byYear.where(byYear['Make'] == make).dropna()
    print(makeYear)
    print('------')


print(byYear['Model Year'].min())
print(byYear['Model Year'].max())


# 

        Make       Model Model Year
445    TESLA  CYBERTRUCK       2024
215    TESLA     MODEL 3       2024
126    TESLA     MODEL 3       2023
3      TESLA     MODEL 3       2022
156    TESLA     MODEL 3       2021
24     TESLA     MODEL 3       2020
27     TESLA     MODEL 3       2019
0      TESLA     MODEL 3       2018
368    TESLA     MODEL 3       2017
785    TESLA     MODEL S       2024
267    TESLA     MODEL S       2023
1139   TESLA     MODEL S       2022
375    TESLA     MODEL S       2021
809    TESLA     MODEL S       2020
82     TESLA     MODEL S       2019
189    TESLA     MODEL S       2018
26     TESLA     MODEL S       2017
146    TESLA     MODEL S       2016
7      TESLA     MODEL S       2015
11     TESLA     MODEL S       2014
99     TESLA     MODEL S       2013
6      TESLA     MODEL S       2012
217    TESLA     MODEL X       2024
98     TESLA     MODEL X       2023
308    TESLA     MODEL X       2022
791    TESLA     MODEL X       2021
202    TESLA     MODEL X    

In [203]:
"""
Is there a correlation between electric range and the city of an EV?

How would we identify this correlation?

Would a lower std (a tighter clustering of data points) indicate this?


"""



"""Dropping 0 values for convince at this stage"""
groupedByCity = data[data['Electric Range'].gt(0)].groupby('City')


print(groupedByCity)
print(groupedByCity.first())
print(groupedByCity.get_group('Aberdeen'))
print('Standard of Deviation', groupedByCity.get_group('Aberdeen')['Electric Range'].std())
print('Mean', groupedByCity.get_group('Aberdeen')['Electric Range'].mean())
print('Mode', groupedByCity.get_group('Aberdeen')['Electric Range'].mode())
print('Quantile', groupedByCity.get_group('Aberdeen')['Electric Range'])






                 Make           Model Model Year Electric Range
City                                                           
Aberdeen         FIAT             500       2013             87
Acme             FORD           C-MAX       2014             19
Addy            TESLA         MODEL 3       2020            308
Airway Heights   FORD          FUSION       2016             19
Alameda          JEEP        WRANGLER       2023             21
...               ...             ...        ...            ...
Yacolt           JEEP  GRAND CHEROKEE       2023             25
Yakima          TESLA         MODEL 3       2018            215
Yarrow Point    TESLA         MODEL S       2015            208
Yelm              BMW              I3       2017             81
Zillah            BMW            330E       2024             20

[619 rows x 4 columns]
             Make          Model Model Year Electric Range      City
32735        FIAT            500       2013             87  Aberdeen
33924 

In [217]:
"""Which county has the greatest variety of EV car models?"""
# Model Count by County

groupedByCity = data[data['Electric Range'].gt(0)].groupby('County')

print(groupedByCity['Model'].unique())


County
Adams           [BOLT EV, MODEL S, C-MAX, FUSION, PACIFICA, LE...
Alameda                                       [WRANGLER, MODEL 3]
Albemarle                                               [MODEL S]
Alexandria                                     [MODEL S, MODEL 3]
Allen                                         [PACIFICA, MODEL S]
                                      ...                        
Whitman         [VOLT, PRIUS PRIME, XC60, ESCAPE, RAV4 PRIME, ...
Wichita                                                 [MODEL 3]
Williamsburg                                               [VOLT]
Williamson                                              [MODEL 3]
Yakima          [MODEL 3, I3, MODEL S, VOLT, SOUL, LEAF, FORTW...
Name: Model, Length: 155, dtype: object
